# AppStore 
As of 2022, Apple's App Store was home to some 1.76 million apps and over 460,000 games. The aim of this exercise is to obtain app rating and review data for select categories for sentiment analysis, product analytics, opportunity discovery, and selection.  Our data acquisition centers on three entities: AppData, Rating, and Review, which are described below.

## AppData
The AppData entity encapsulates the core data for each app and is defined as follows.  

| #  | attribute     | type  | description                                  | API Field         |
|----|---------------|-------|----------------------------------------------|-------------------|
| 1  | id:           | int   | Unique Apple App Identifier                  | trackId           |
| 2  | name:         | str   | Name of the app.                             | trackName         |
| 3  | description:  | str   | Description                                  | description       |
| 4  | category_id:  | int   | Four digit category identifier               | primaryGenreId    |
| 5  | category:     | str   | Category name                                | primaryGenreName  |
| 6  | price:        | float | Cost of the app                              | price             |
| 7  | rating:       | float | The user average rating                      | averageUserRating |
| 8  | ratings:      | int   | The rating count                             | userRatingCount   |
| 9  | developer_id: | int   | The app developer identifier                 | artistId          |
| 10 | developer:    | str   | The app developer name                       | artistName        |
| 11 | released:     | str   | The date of initial release                  | releaseDate       |
| 12 | source:       | str   | The host from which the data were obtained.  | itunes.apple.com  |

The data acquisition pipeline will obtain app data for the following categories and persist them in an RDBMS table.

1. business
2. education
3. entertainment
4. health
5. lifestyle
6. medical
7. productivity
9. social_networking


### Imports

In [1]:
from aimobile.service.appstore.controller import AppStoreAppController, AppStoreReviewController
from aimobile.container import AIMobileContainer

In [2]:
TERMS = ["business", "education", "entertainment", "health", "lifestyle", "medical", "productivity", "social_networking"]
CATEGORIES = [6000, 6017, 6016, 6013, 6012, 6020, 6007, 6005]

### Dependencies

In [3]:
container = AIMobileContainer()
container.init_resources()
container.wire(packages=["aimobile.service.appstore"])

### AppData Scraper
AppStoreAppController object iterates through the above TERMs, engaging a scraper to extract the app data described above from the App Store. The results are persisted in an RDBMS and archived.

In [4]:
controller = AppStoreAppController()
controller.scrape(terms=TERMS)
controller.summarize()


[04/22/2023 03:27:59 PM] [INFO] [AppStoreAppController] [scrape] : 

Appstore AppData Scraped Status is Complete. Skipping App Store App Data Scraping Operation.


,Category,App Count,Average Rating,Average Rating Count,Total Rating Count
0,Medical,138680,1.54,203.74,28255057
1,Health & Fitness,67879,2.74,"1,189.69",80755198
2,Social Networking,63761,1.78,994.00,63378258
3,Education,27807,2.56,"2,281.81",63450186
4,Business,27534,2.68,"2,321.21",63912296
5,Lifestyle,21507,3.09,"3,687.58",79308887
6,Games,19887,4.09,"12,061.64",239869878
7,Productivity,15179,3.11,"4,786.31",72651335
8,Utilities,12559,2.80,"3,567.25",44801098
9,Entertainment,10278,3.33,"10,063.51",103432717


### AppData Exploration

In [5]:
repo = container.appstore.appdata_repo()
df = repo.getall()

In [6]:
df.info()
df.head()
df.tail()
df.sample(10)
df.loc[df['ratings']< 100].sort_values(by='rating', ascending=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450850 entries, 0 to 450849
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            450850 non-null  int64  
 1   name          450850 non-null  object 
 2   description   450850 non-null  object 
 3   category_id   450850 non-null  int64  
 4   category      450850 non-null  object 
 5   price         450850 non-null  float64
 6   developer_id  450850 non-null  int64  
 7   developer     450850 non-null  object 
 8   rating        450850 non-null  float64
 9   ratings       450850 non-null  int64  
 10  released      450850 non-null  object 
 11  source        450850 non-null  object 
dtypes: float64(2), int64(4), object(6)
memory usage: 41.3+ MB


,id,name,description,category_id,category,price,developer_id,developer,rating,ratings,released,source
0,1386412985,WhatsApp Business,A simple tool for businesses to talk to their ...,6000,Business,0.00,310634000,WhatsApp Inc.,4.74,579101,2019-04-05T05:53:41Z,itunes.apple.com
1,514643583,Meta Business Suite,Grow your business and connect with more peopl...,6000,Business,0.00,284882218,"Meta Platforms, Inc.",4.64,225999,2012-05-22T00:21:31Z,itunes.apple.com
2,389801252,Instagram,Bringing you closer to the people and things y...,6008,Photo & Video,0.00,389801255,"Instagram, Inc.",4.73,24573612,2010-10-06T08:12:41Z,itunes.apple.com
3,711923939,Cash App,"Cash App is the easy way to send, spend, save,...",6015,Finance,0.00,335393791,"Square, Inc.",4.78,4658636,2013-10-16T07:51:06Z,itunes.apple.com
4,951937596,Microsoft Outlook,Outlook lets you bring all your email accounts...,6007,Productivity,0.00,298856275,Microsoft Corporation,4.77,5589383,2015-01-29T05:22:59Z,itunes.apple.com


,id,name,description,category_id,category,price,developer_id,developer,rating,ratings,released,source
450845,1636736385,OrthoExamine,OrthoExamine ist eine APP die dem Anwender sch...,6020,Medical,3.99,1636736747,orthoapps UG,0.00,0,2022-12-08T08:00:00Z,itunes.apple.com
450846,6444037473,TROPA,TROPA- это ваш набор инструментов для улучшени...,6020,Medical,0.00,1651597807,Vitaly Russkikh,0.00,0,2022-12-10T08:00:00Z,itunes.apple.com
450847,1608859729,Augustinus Apotheke Vorau,Die Augustinus Apotheke Vorau-App ist dein dig...,6013,Health & Fitness,0.00,1608859731,Augustinus Apotheke Mag. pharm. Sallegger KG,0.00,0,2022-02-17T08:00:00Z,itunes.apple.com
450848,977242433,必勝カコもん作業療法士,作業療法士国家試験対策用分野別問題集。\n過去８年分の国家試験問題をもとに構成。現役教員によ...,6020,Medical,19.99,478753862,"RoundFlat, Inc.",0.00,0,2015-04-04T06:40:48Z,itunes.apple.com
450849,1638895537,Hessisches Ärzteblatt Digital,"Das Hessische Ärzteblatt, amtliches Mitteilung...",6020,Medical,0.00,393515268,Deutscher Aerzte-Verlag GmbH,0.00,0,2022-08-09T07:00:00Z,itunes.apple.com


,id,name,description,category_id,category,price,developer_id,developer,rating,ratings,released,source
433227,1536357440,医療法人 石川病院,さいたま市浦和区に大正11年からある医療法人石川病院です。\nアプリからボタン1つで病院に電...,6020,Medical,0.00,1536357442,医療法人石川病院,0.00,0,2020-11-01T07:00:00Z,itunes.apple.com
305416,1661214922,Fetes Tv,Fetes Tv Is a Global Social Media and World M...,6011,Music,0.00,1634705965,Lindoff Lindsay,5.00,2,2023-01-20T08:00:00Z,itunes.apple.com
62520,790049340,myAir Health,"At myAir, we provide you with timely, personal...",6013,Health & Fitness,0.00,790049343,myAir,3.50,6,2014-01-31T07:35:33Z,itunes.apple.com
57833,1403059908,Carb & Keto Manager - Aspire,Do you aspire to lose weight & live healthier?...,6013,Health & Fitness,0.00,1403059907,"Honeycrisp Technologies, Inc.",4.68,385,2018-08-04T02:28:16Z,itunes.apple.com
271521,1564649902,Eva,Features:\nSave you more.\n-Free shipping for ...,6000,Business,0.00,1579670348,Safya Marri,5.00,2,2021-06-16T07:00:00Z,itunes.apple.com
346277,1621781897,Igreja Vida Acailândia,"Olá, seja bem-vindo este é App oficial da Igre...",6005,Social Networking,0.00,1621781899,IGREJA VIDA EM ACAILANDIA - MA,0.00,0,2022-04-30T07:00:00Z,itunes.apple.com
349603,1666499081,Club de Baloncesto Leopardos,Official App of Club de Baloncesto Leopardos. ...,6004,Sports,0.00,1282527621,Clupik,0.00,0,2023-01-19T08:00:00Z,itunes.apple.com
52512,1452973510,LUDO‎,Ludo is fun to play a multiplayer board game t...,6014,Games,0.00,1294190633,Yarsa Labs,4.72,1022,2019-02-15T11:07:18Z,itunes.apple.com
76084,1095999436,RVC Pet Diabetes App,Pets with diabetes can be treated very well re...,6020,Medical,0.00,686454449,Royal Veterinary College,3.27,11,2016-03-28T23:50:40Z,itunes.apple.com
122430,561171830,Easy Legs LE,Get a great leg workout in just 5 minutes! \n\...,6013,Health & Fitness,0.00,383811733,lolo,5.00,2,2012-09-19T01:40:33Z,itunes.apple.com


,id,name,description,category_id,category,price,developer_id,developer,rating,ratings,released,source
379510,1047761003,CPUP Hip Score,CPUP is a follow-up program and national quali...,6020,Medical,0.00,1455942361,Gunnar Hagglund,5.00,1,2015-10-18T19:41:30Z,itunes.apple.com
151729,1439490170,VieCure Mobile,VieCure continues to simplify cancer's complex...,6020,Medical,0.00,1131712436,Viviphi LTD,5.00,5,2018-10-21T05:58:34Z,itunes.apple.com
324996,1602993186,York Uni,This app is meant to help you with navigating ...,6017,Education,0.00,1532440924,Aayush Pokharel,5.00,1,2022-09-03T07:00:00Z,itunes.apple.com
151742,1050103022,H.U. General de Villalba,Esta aplicación ha sido diseñada para facilita...,6020,Medical,0.00,971821249,idcsalud,5.00,1,2015-12-05T01:12:39Z,itunes.apple.com
259308,1484111618,Quicklert,Quicklert application securely receives messag...,6000,Business,0.00,1199099659,Quicklert Inc.,5.00,2,2019-12-10T08:00:00Z,itunes.apple.com
...,...,...,...,...,...,...,...,...,...,...,...,...
285017,1555415773,AP POLICE SEVA,AP POLICE SEVA is a step towards the safety an...,6005,Social Networking,0.00,1555411967,AP POLICE TECHNICAL SERVICES,0.00,0,2021-03-06T08:00:00Z,itunes.apple.com
285019,6443999812,GUYUK Metaverse,"The GUYUK metaverse will provide tourism, heri...",6014,Games,0.00,1651357801,Byambasuren Ganbold,0.00,0,2022-11-16T08:00:00Z,itunes.apple.com
285023,1492505590,Hydra Social,Hydra empowers you to replace the clutter of s...,6005,Social Networking,0.00,1389887758,HAIDARCORP LLC,0.00,0,2022-06-01T07:00:00Z,itunes.apple.com
285024,966831550,respon -レスポン-,respon‐レスポン‐は、無料で使える【リアルタイムアンケート＆クリッカーアプリ】です。 ...,6017,Education,0.00,1441360548,"respon, Inc.",0.00,0,2021-06-13T07:00:00Z,itunes.apple.com
